In [1]:
import torch

if torch.cuda.is_available():    
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [2]:
import pandas as pd
import math
from sklearn import preprocessing

file = "hasoc_2020_de_train_new_b.xlsx"
file_test = "german_test_1509.csv"


df_train = pd.read_excel(file,index_col=0)

df_train = df_train.dropna()

df_test = pd.read_csv(file_test)

task = 'task2'
task_2019 = 'task_2'

#2019 datasets also

file_2019_1 = pd.read_csv("2019/german_dataset/german_dataset/hasoc_de_test_gold.tsv",sep='\t')
file_2019_2 = pd.read_csv("2019/german_dataset/german_dataset/german_dataset.tsv",sep="\t")


sentences_2019_1 = list(file_2019_1['text'].values)
sentences_2019_2 = list(file_2019_2['text'].values)

labels_2019_1 = list(file_2019_1[task_2019].values)
labels_2019_2 = list(file_2019_2[task_2019].values)

print(len(df_train))
print(df_train.head())

total_sentences = list(df_train['text'].values)
total_labels = list(df_train[task].values)

total_sentences.extend(sentences_2019_1)
total_sentences.extend(sentences_2019_2)

total_labels.extend(labels_2019_1)
total_labels.extend(labels_2019_2)

test_sentences = list(df_test['text'].values)
test_labels = list(df_test[task].values)

def clean_text(sentences):
    for index,line in enumerate(sentences):
        if "\n" in line:
            sentences[index] = line.replace("\n","")
    return sentences
        
total_sentences = clean_text(total_sentences)
test_sentences = clean_text(test_sentences)

def clean_labels(labels):
    new_list= []
    for value in labels:
        new_list.append(value.strip())
    return new_list

total_labels = clean_labels(total_labels)
test_labels = clean_labels(test_labels)

le = preprocessing.LabelEncoder()
le.fit(total_labels)
encoded_labels = le.transform(total_labels)
encoded_test_labels = le.transform(test_labels)
print(set(encoded_labels))

print(len(total_sentences),len(encoded_labels),len(test_sentences),len(encoded_test_labels))

print(df_test)

2373
                                                                  text task1  \
tweet_id                                                                       
1133388798925189122  Deutsche rothaarige porno reife deutsche fraue...   NOT   
1127134592517980161  RT @NDRinfo: Die deutsche Klimaaktivistin Luis...   NOT   
1128897106171842560  @ruhrbahn jeden Morgen eine neue „Fahrzeugstör...   NOT   
1123576753199484928  @Junge_Freiheit Die Inkas hatten sich schon dä...   NOT   
1128743783393312768  RT @technosteron: leute die 'boar' schreiben l...   HOF   

                    task2                  ID  
tweet_id                                       
1133388798925189122  NONE  hasoc_2020_de_2684  
1127134592517980161  NONE  hasoc_2020_de_1042  
1128897106171842560  NONE   hasoc_2020_de_774  
1123576753199484928  NONE   hasoc_2020_de_559  
1128743783393312768  PRFN  hasoc_2020_de_1969  
{0, 1, 2, 3}
7042 7042 526 526
                tweet_id                                           

In [3]:
from transformers import BertTokenizer


tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

max_length = 0
for sentence in total_sentences:
    #print(sentence)
    length = len(tokenizer.tokenize(sentence))
    if length > max_length:
        max_length  = length
print("max token length is: ",max_length)
# max token length obtained is 50
# bert tokens are limited to 514 bytes.

max token length is:  175


In [4]:
def encoder_generator(sentences,labels):
    
    sent_index = []
    input_ids = []
    attention_masks =[]

    for index,sent in enumerate(sentences):
        
        sent_index.append(index)
        
        encoded_dict = tokenizer.encode_plus(sent,
                                             add_special_tokens=True,
                                             max_length=max_length,
                                             pad_to_max_length=True,
                                             truncation = True,
                                             return_attention_mask=True,
                                             return_tensors='pt')
        input_ids.append(encoded_dict['input_ids'])

        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids,dim=0)
    attention_masks = torch.cat(attention_masks,dim=0)
    labels = torch.tensor(labels)
    sent_index = torch.tensor(sent_index)

    return sent_index,input_ids,attention_masks,labels

sent_index,input_ids,attention_masks,encoded_label_tensors = encoder_generator(total_sentences,encoded_labels)
test_sent_index,test_input_ids,test_attention_masks,encoded_test_label_tensors = encoder_generator(test_sentences,encoded_test_labels)
print('Original: ', total_sentences[0])
print('Token IDs:', input_ids[0])
#print(encoded_label_tensors)
#print(encoded_test_label_tensors)

Original:  Deutsche rothaarige porno reife deutsche frauen porno. Deutsche politessen pornos porno deutsch inzets. https://t.co/xAag87Y0Jd
Token IDs: tensor([   101,  15389,  64354,  55200,  45854,  10183,  10343,  14243,  14601,
         17486,  10628,  25733,  10183,  10343,    119,  15389,  91929, 100319,
         10115,  10183,  14386,  10183,  10343,  24722,  10106,  17931,  10107,
           119,  14120,    131,    120,    120,    188,    119,  11170,    120,
           192,  10738,  14520,  11396,  11305,  14703,  10929,  15417,  10162,
           102,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,   

In [5]:
from torch.utils.data import TensorDataset,random_split

dataset = TensorDataset(input_ids,attention_masks,encoded_label_tensors)
test_dataset = TensorDataset(test_sent_index,test_input_ids,test_attention_masks,encoded_test_label_tensors)

train_size = int(0.75*len(dataset))

val_size = len(dataset)-train_size

train_dataset,val_dataset = random_split(dataset,[train_size,val_size])

print('train data samples is {}'.format(len(train_dataset)))
print("valid data samples is {}".format(len(val_dataset)))
print("test data samples is {}".format(len(test_dataset)))

train data samples is 5281
valid data samples is 1761
test data samples is 526


In [6]:
from torch.utils.data import DataLoader,RandomSampler,SequentialSampler

bs=8

train_data_loader = DataLoader(train_dataset,
                              sampler=RandomSampler(train_dataset),
                              batch_size=bs)
valid_data_loader = DataLoader(val_dataset,
                              sampler=SequentialSampler(val_dataset),
                              batch_size=bs)
test_data_loader = DataLoader(test_dataset,
                            sampler=SequentialSampler(test_dataset),
                            batch_size=bs)

In [7]:
from transformers import BertForSequenceClassification, AdamW

model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased',
                                                     num_labels=len(le.classes_),
                                                     output_attentions=False,
                                                     output_hidden_states=False,
                                                     )
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [8]:
optimizer = AdamW(model.parameters(),lr=2e-5,eps=1e-8)

from transformers import get_linear_schedule_with_warmup

epochs=10
total_steps = len(train_data_loader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer,
                                           num_warmup_steps=0,
                                           num_training_steps=total_steps)

In [9]:
import numpy as np

def predictions_labels(preds,labels):
    pred = np.argmax(preds,axis=1).flatten()
    label = labels.flatten()
    return pred,label

In [10]:
import random
import numpy as np
import time
from sklearn.metrics import classification_report,accuracy_score,f1_score

total_t0 = time.time()

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [11]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]])

def predictions_labels(preds,labels):
    pred = np.argmax(preds,axis=1).flatten()
    label = labels.flatten()
    return pred,label

In [12]:
def train():
    total_train_loss = 0
    total_train_acc = 0
    
    model.train() # set model in train mode for batchnorm and dropout layers in bert model
    
    for step,batch in enumerate(train_data_loader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        model.zero_grad()
            
        loss,logits = model(b_input_ids,
                            attention_mask=b_input_mask,
                            labels=b_labels.long())
            
        total_train_loss+=loss.item()
        total_train_acc+=categorical_accuracy(logits,b_labels).item()
            
        loss.backward()
            
        torch.nn.utils.clip_grad_norm_(model.parameters(),1.0)
            
        optimizer.step()
            
        scheduler.step() #go ahead and update the learning rate
            
    avg_train_loss = total_train_loss/len(train_data_loader)
    avg_train_acc = total_train_acc/len(train_data_loader)
    
    return avg_train_loss,avg_train_acc

In [13]:
def evaluate():
    model.eval()
        
    total_eval_accuracy = 0
    total_eval_loss = 0
    number_of_eval_steps= 0
    
    all_true_labels = []
    all_pred_labels = []

    for batch in valid_data_loader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():

            loss, logits = model(b_input_ids,
                                attention_mask= b_input_mask,
                                labels = b_labels.long())
        total_eval_loss+=loss.item()

        logits = logits.detach().cpu().numpy()

        label_ids = b_labels.to('cpu').numpy()

        pred,true = predictions_labels(logits,label_ids)
        
        all_pred_labels.extend(pred)
        all_true_labels.extend(true)

    print(classification_report(all_pred_labels,all_true_labels))
    avg_val_accuracy = accuracy_score(all_pred_labels,all_true_labels)
    macro_f1_score = f1_score(all_pred_labels,all_true_labels,average='macro')
    
    avg_val_loss = total_eval_loss/len(valid_data_loader)

    print("accuracy = {0:.2f}".format(avg_val_accuracy))
    
    return avg_val_loss,avg_val_accuracy,macro_f1_score

In [14]:
import time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [15]:
epochs = 10

best_macro_f1 = float('0')

for epoch in range(epochs):
    
    start_time = time.time()
    train_loss,train_acc = train()
    valid_loss,valid_acc,macro_f1 = evaluate()
    
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if macro_f1 > best_macro_f1:
        best_macro_f1 = macro_f1
        torch.save(model,'model_german_task_b.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

C:\Users\suman\Miniconda3\envs\py3_env\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\suman\Miniconda3\envs\py3_env\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.97      0.89      0.93      1602
           2       0.10      0.61      0.18        18
           3       0.71      0.54      0.61       141

   micro avg       0.86      0.86      0.86      1761
   macro avg       0.45      0.51      0.43      1761
weighted avg       0.94      0.86      0.89      1761

accuracy = 0.86
Epoch: 01 | Epoch Time: 3m 20s
	Train Loss: 0.606 | Train Acc: 83.23%
	 Val. Loss: 0.538 |  Val. Acc: 85.69%
              precision    recall  f1-score   support

           0       0.01      0.50      0.03         2
           1       0.98      0.88      0.93      1637
           2       0.09      1.00      0.16         9
           3       0.64      0.60      0.62       113

   micro avg       0.86      0.86      0.86      1761
   macro avg       0.43      0.75      0.43      1761
weighted avg       0.95      0.86      0.90      1761

accur

In [15]:
del model
import gc
gc.collect()

model = torch.load('model_german_task_b.pt')
model = model.to(device)

In [16]:
def evaluate_test():
    model.eval()
        
    total_eval_accuracy = 0
    total_eval_loss = 0
    number_of_eval_steps= 0
    
    all_true_labels = []
    all_pred_labels = []
    
    all_sentence_id=[]

    for batch in test_data_loader:
        b_sentence_id = batch[0].to(device)
        b_input_ids = batch[1].to(device)
        b_input_mask = batch[2].to(device)
        b_labels = batch[3].to(device)

        sent_ids = b_sentence_id.to('cpu').numpy()
        all_sentence_id.extend(sent_ids)
        
        with torch.no_grad():

            loss, logits = model(b_input_ids,
                                attention_mask= b_input_mask,
                                labels = b_labels.long())
        total_eval_loss+=loss.item()

        logits = logits.detach().cpu().numpy()

        label_ids = b_labels.to('cpu').numpy()
        

        pred,true = predictions_labels(logits,label_ids)
        
        all_pred_labels.extend(pred)
        
        all_true_labels.extend(true)

    print(classification_report(all_pred_labels,all_true_labels))
    avg_val_accuracy = accuracy_score(all_pred_labels,all_true_labels)
    
    avg_val_loss = total_eval_loss/len(valid_data_loader)

    print("accuracy = {0:.2f}".format(avg_val_accuracy))
    
    return avg_val_loss,avg_val_accuracy,all_sentence_id,all_pred_labels

valid_loss,valid_acc,all_sentence_id,all_pred_labels = evaluate_test()

              precision    recall  f1-score   support

           0       0.17      0.36      0.23        11
           1       0.93      0.86      0.89       407
           2       0.06      0.40      0.10         5
           3       0.75      0.64      0.69       103

   micro avg       0.80      0.80      0.80       526
   macro avg       0.48      0.57      0.48       526
weighted avg       0.87      0.80      0.83       526

accuracy = 0.80


In [17]:
pred_labels = list(le.inverse_transform(all_pred_labels))

In [18]:
from collections import OrderedDict
dict_index_y_value = {}

for index,y_value in zip(all_sentence_id,pred_labels):
    dict_index_y_value[index] = y_value

od = OrderedDict(sorted(dict_index_y_value.items()))

sorted_y_predicts = []
for k,v in od.items():
    sorted_y_predicts.append(v)

In [19]:
test_tweet_id = list(df_test['tweet_id'].values)
test_ID = list(df_test['ID'].values)

test_final = pd.DataFrame(zip(test_tweet_id,sorted_y_predicts,test_ID),columns=['tweet_id',task,'ID'])

if task == 'task1':
    test_final.to_csv("submission_DE_A.csv",index=False)

if task == 'task2':
    test_final.to_csv("submission_DE_B.csv",index=False)

In [20]:
del model
import gc
gc.collect()

0